In [11]:
import torch
import torch.nn as nn
from models.MMR import MMR
from models.MMR import TDB
from models.MMR import MMR_losses
from models.image_2_recipe import Image2Recipe
from models.image_encoder import Image_Encoder
from models.recipe_encoder import RecipeEncoder
# from models.Image2Recipe import Image2Recipe

# Define dummy data parameters
batch_size = 8  # Adjust as needed
seq_len = 10  # Length of the token sequence, adjust as needed
hidden_dim = 1024  # Size of the feature vectors
num_heads = 4  # Number of attention heads
ITEM_lyrs = 2  # Number of ITEM layers
MTD_lyrs = 2  # Number of MTD layers
projection_dim = 512  # Projection dimension



In [13]:
# Create dummy data (projected recipe and image embeddings)
image_encoder = Image_Encoder(self.device, self.clip_model, num_classes).to(self.device)
recipe_encoder = RecipeEncoder(self.device, self.vocab_size, self.max_len).to(self.device)
mmr = MMR(input_size=self.image_encoder.clip_model.config.projection_dim, device=self.device)
# num_heads=num_heads, ITEM_lyrs=ITEM_lyrs, MTD_lyrs=MTD_lyrs, projection_dim=projection_dim
model = Image2Recipe(self.image_encoder, self.recipe_encoder, self.mmr).to(self.device)


NameError: name 'self' is not defined

In [2]:
        
recipe_tokens = torch.randn(batch_size, seq_len, hidden_dim)  # Shape: [batch_size, seq_len, hidden_dim]
image_tokens = torch.randn(batch_size, seq_len, hidden_dim)  # Shape: [batch_size, seq_len, hidden_dim]
# Create labels for testing
labels = torch.randint(0, 2, (batch_size,)).float()  # Random binary labels for instance loss calculation


In [3]:
# Instantiate the MMR model
mmr_model = MMR(hidden_dim=hidden_dim, num_heads=num_heads, ITEM_lyrs=ITEM_lyrs, MTD_lyrs=MTD_lyrs, projection_dim=projection_dim)


In [4]:
print(recipe_tokens.shape)
print(image_tokens.shape)
print(labels.shape)

torch.Size([8, 10, 1024])
torch.Size([8, 10, 1024])
torch.Size([8])


In [5]:
# Forward pass through the model
logits = mmr_model(recipe_tokens, image_tokens)


In [6]:
print(logits.shape)

torch.Size([8])


In [7]:
print(labels)

tensor([0., 0., 0., 0., 0., 0., 1., 1.])


In [8]:
loss = MMR_losses(margin=1.0, instance_weight=1.0, sem_weight=1.0, itm_weight=1.0)
semantic_loss = loss.instance_semantic_loss(image_tokens, recipe_tokens, labels, margin=1, mode='semantic')
instance_loss = loss.instance_semantic_loss(image_tokens, recipe_tokens, labels, margin=1, mode='instance')
itm_loss = loss.itm_loss(logits, labels)

loss = loss.total_loss(labels, image_tokens, recipe_tokens, logits)

# Print the outputs for verification
print("Logits:", logits)
print("Instance Loss:", instance_loss.item())
print("Semantic Loss:", semantic_loss.item())

Logits: tensor([6.4057e-01, 2.3168e-04, 1.3737e-01, 8.5390e-02, 2.2100e-02, 9.6236e-01,
        3.5823e-01, 9.9984e-01], grad_fn=<MeanBackward1>)
Instance Loss: 4.0254292488098145
Semantic Loss: 2.4990320205688477


In [9]:
print(itm_loss)

tensor(0.6986, grad_fn=<BinaryCrossEntropyBackward0>)


In [10]:
print(loss)

tensor(7.2231, grad_fn=<AddBackward0>)
